In [ ]:
!pip install textacy beautifulsoup4 wikipedia --quiet
!python --version

Python 3.7.10


In [ ]:
import re
import requests
import json
import wikipedia
import string
import numpy as np
import nltk
from bs4 import BeautifulSoup
import multiprocessing
from random import randrange
from collections import defaultdict
from gensim.models.word2vec import Word2Vec
from gensim.models.phrases import Phraser, Phrases

In [ ]:
PUNCT = list(string.punctuation)
STOP_WORDS = set(["the"])
IGNORE_WORDS = {"also", "not", "why", "does", "how", "many", "so", "much", "where", "when", "which", "from", "all", "am", "an", "and", "another", "any", "are", "as", "at", "be",
              "been", "being", "but", "by", "came", "can", "come", "did", "do", "for", "get", "got", "has", "had", "he",
              "have", "her", "here", "him", "himself", "his", "if", "in", "into", "is", "it", "like", "me", "my",
              "of", "on", "or", "other", "our", "out", "over", "see", "still", "such", "take", "than", "that", "the",
              "their", "them", "then", "there", "these", "they", "this", "those", "through", "to", "too", "up", "was",
              "way", "we", "well", "while", "with", "would", "you", "your", "a", "i", "will", "com", "may", "every",
              "using", "just", "need", "want", "years", "great", "good", "privacy", "next", "know", "found", "add",
              "even", "use", "one", "something", "choice", "some", "more", "away", "really", "put", "instead", "start"}

In [ ]:
nltk.download('punkt', quiet=True)

True

In [ ]:
cores = multiprocessing.cpu_count()
cores

2

In [ ]:
text = """how to cook bacon in the oven.
how to cook bacon in the oven without a rack.
how to cook bacon in the oven without making a mess.
how to cook bacon in the oven with foil.
how to cook bacon in the oven uk.
how to cook bacon in the oven without parchment paper.
how to cook bacon in the oven on parchment paper.
how to cook bacon in the oven on a rack.
how to cook bacon in the oven rachael ray.
how to cook bacon in the oven at 350.
how to cook spaghetti squash.
how to cook spaghetti squash in the oven.
how to cook spaghetti squash whole.
how to cook spaghetti squash in instant pot.
how to cook spaghetti squash in microwave.
how to cook spaghetti squash fast.
how to cook spaghetti squash in air fryer.
how to cook spaghetti squash without an oven.
how to cook spaghetti squash in crock pot.
how to cook spaghetti squash noodles.
how to cook lobster tail.
how to cook lobster tail in oven.
how to cook lobster tail on the grill.
how to cook lobster tail boil.
how to cook lobster tail in air fryer.
how to cook lobster tail from frozen.
how to cook lobster tails from costco.
how to cook lobster tail on stove.
how to cook lobster tail steam.
how to cook lobster tails for lobster rolls.
how to cook rice.
how to cook rice on stove.
how to cook rice in instant pot.
how to cook rice in a rice cooker.
how to cook rice noodles.
how to cook rice in microwave.
how to cook rice in a crock pot.
how to cook riced cauliflower.
how to cook rice in the oven.
how to cook rice for fried rice.
how to cook salmon.
how to cook salmon in the oven.
how to cook salmon in a pan.
how to cook salmon in air fryer.
how to cook salmon on the grill.
how to cook salmon with skin.
how to cook salmon patties.
how to cook salmon fillet in oven.
how to cook salmon burgers.
how to cook salmon steaks.
how to cook scallops.
how to cook scallops in oven.
how to cook scallops wrapped in bacon.
how to cook scallops on the grill.
how to cook scallops on stove.
how to cook scallops from frozen.
how to cook scallops in air fryer.
how to cook scallops for pasta.
how to cook scallops and shrimp.
how to cook scallops in shell.
how to cook prime rib.
how to cook prime rib steak.
how to cook prime rib on the grill.
how to cook prime rib steak in oven.
how to cook prime rib in crock pot.
how to cook prime rib steak on the grill.
how to cook prime rib on a pellet grill.
how to cook prime rib in air fryer.
how to cook prime rib in a roaster.
how to cook prime rib roast boneless.
how to cook crab legs.
how to cook crab legs in the oven.
how to cook crab legs on the grill.
how to cook crab legs from frozen.
how to cook crab legs boil.
how to cook crab legs in instant pot.
how to cook crab legs in air fryer.
how to cook crab legs from costco.
how to cook crab legs and lobster tails.
how to cook crab legs with old bay.
how to cook salmon in the oven with foil.
how to cook salmon in the oven at 350.
how to cook salmon in the oven with skin.
how to cook salmon in the oven at 400.
how to cook salmon in the oven without foil.
how to cook salmon in the oven from frozen.
how to cook salmon in the oven youtube.
how to cook salmon in the oven in a cast iron skillet.
how to cook salmon in the oven on a cedar plank.
how to cook filet mignon.
how to cook filet mignon on stove.
how to cook filet mignon on the grill.
how to cook filet mignon in air fryer.
how to cook filet mignon in pan.
how to cook filet mignon wrapped in bacon.
how to cook filet mignon roast.
how to cook filet mignon in ninja foodi.
how to cook filet mignon in oven only.
how to cook filet mignon on traeger.
how long to cook a turkey.
how long to cook a turkey at 325.
how long to cook a turkey at 350.
how long to cook a turkey in the oven.
how long to cook a turkey in a bag.
how long to cook a turkey breast per pound.
how long to cook a turkey breast in the oven.
how long to cook a turkey burger.
how long to cook a turkey in a roaster.
how long to cook a turkey meatloaf."""

In [ ]:
sentences_list = nltk.sent_tokenize(text)

In [ ]:
def remove_not_valid_chars(word, include_space=True):
  word_lowercase = word.lower()
  if include_space:
    return re.sub(r'[^a-zA-Z0-9- ]+', '', word_lowercase)
  return re.sub(r'[^a-zA-Z0-9-]+', '', word_lowercase)

In [ ]:
def most_frequent_words(phraser, sents, num, min_word_len=0, max_word_len=100):
    word_freq = defaultdict(int)

    for sent in phraser[sents]:
        for i in sent:
            l = len(i.split("_"))
            if i not in STOP_WORDS and l >= min_word_len and l <= max_word_len:
              word_freq[i] += 1
    
    words = []
    for k in sorted(word_freq, key=word_freq.get, reverse=True)[:num]:        
      words.append(k)
        
    return words

In [ ]:
sentences = []

for sent in sentences_list:
  clean_words = []
  words = nltk.word_tokenize(sent)

  for word in words:
      w = remove_not_valid_chars(word, False)
      if w and len(w) > 1 and not w.isdigit() and w not in PUNCT and w not in STOP_WORDS:
          clean_words.append(w)

  if len(clean_words) > 2:
      sentences.append(clean_words)

In [ ]:
sentences[0]

['how', 'to', 'cook', 'bacon', 'in', 'oven']

In [ ]:
bigram = Phrases(sentences, min_count=2, threshold=1, common_terms=STOP_WORDS)
bigram_model = Phraser(bigram)
trigram = Phrases(bigram[sentences], min_count=2, threshold=10, common_terms=STOP_WORDS)
trigram_model = Phraser(trigram)
phraser = trigram_model[bigram_model[sentences]]

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
freq = most_frequent_words(trigram_model, sentences, 1000, 1)
len(freq)

98

In [ ]:
"\n".join(freq)

'how\nto\ncook\nin_oven\nin\nsalmon\ncook_bacon\nspaghetti\ncook_lobster\nscallops\nprime\nrib\ncrab\nlegs\nfilet\nmignon\nlong\nturkey\ncook_rice\ntail\nair\nfryer\non_grill\npot\nfrom\nwithout\nwith\non\nat\nsquash\nsquash_in\nfrozen\non_stove\nfoil\ninstant\ncrock\ntails\nfor\nsteak\nrack\nparchment\npaper\noven\nmicrowave\nnoodles\nboil\ncostco\nlobster\nrice\npan\nskin\nwrapped\nbacon\nand\nroaster\nroast\nbreast\nmaking\nmess\nuk\nrachael\nray\nwhole\nfast\nan\nsteam\nrolls\ncooker\nriced\ncauliflower\nfried\npatties\nfillet\nburgers\nsteaks\npasta\nshrimp\nshell\npellet\ngrill\nboneless\nold\nbay\nyoutube\ncast\niron\nskillet\ncedar\nplank\nninja\nfoodi\nonly\ntraeger\nbag\nper\npound\nburger\nmeatloaf'

In [ ]:
w2v_model = Word2Vec(
    size=200,    
    min_count=1,
    workers=2,    
)

In [ ]:
w2v_model.build_vocab(phraser)

In [ ]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=50)

(13002, 37250)

In [ ]:
len(w2v_model.wv.vocab)

107

In [ ]:
popular = most_frequent_words(trigram, sentences, 100, 1, 3)

all_words = []
for pop in popular:
    all_words.append(pop)
    if pop not in STOP_WORDS and pop not in IGNORE_WORDS and pop in w2v_model.wv:
        w2v_words = w2v_model.wv.most_similar(pop, topn=10)
        for w in w2v_words:
            if w not in STOP_WORDS and w not in IGNORE_WORDS:
                all_words.append(w[0])

all_words = set(all_words)  

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
all_words

{'air',
 'air_fryer',
 'an',
 'and',
 'at',
 'at_how',
 'bacon',
 'bag',
 'bay',
 'boil',
 'boneless',
 'breast',
 'burger',
 'burgers',
 'cast',
 'cauliflower',
 'cedar',
 'cook',
 'cook_bacon',
 'cook_crab_legs',
 'cook_filet',
 'cook_filet_mignon',
 'cook_lobster',
 'cook_lobster_tail',
 'cook_prime',
 'cook_rice',
 'cook_salmon',
 'cook_spaghetti_squash',
 'cook_spaghetti_squash_in',
 'cooker',
 'costco',
 'crab',
 'crock',
 'fast',
 'filet',
 'fillet',
 'foil',
 'foodi',
 'for',
 'fried',
 'from',
 'from_frozen',
 'frozen',
 'fryer',
 'grill',
 'how',
 'how_to',
 'in',
 'in_air_fryer',
 'in_crock',
 'in_instant',
 'in_oven',
 'instant',
 'instant_pot',
 'iron',
 'legs',
 'lobster',
 'lobster_tails',
 'long',
 'long_to',
 'making',
 'meatloaf',
 'mess',
 'microwave',
 'mignon',
 'mignon_on',
 'ninja',
 'noodles',
 'old',
 'on',
 'on_grill',
 'on_stove',
 'only',
 'oven',
 'oven_with',
 'oven_without',
 'pan',
 'paper',
 'parchment',
 'pasta',
 'patties',
 'pellet',
 'per',
 'plank'